In [15]:
from Article import Article
from Utils import parse_text, parse_tags, parse_saves, parse_views, is_ads
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import csv

In [2]:
# id
DATA_ID = 'data-story-id'
# title
CLASS_TITLE = 'story__title-link'
# author id
DATA_AUTHOR_ID = 'data-author-id'
# author name
CLASS_AUTHOR = 'story__user-link user__nick'
# author name
DATA_AUTHOR_NAME = 'data-name'
# comments count
DATA_COMMENTS_COUNT = 'data-comments'
# rating info
DATA_RATING = 'data-rating'
# meta rating info
DATA_META_RATING = 'data-meta-rating'
# time
DATA_TIME = 'data-timestamp'
# tag info
CLASS_TAG = 'tags__tag'
# tag name
DATA_TAG = 'data-tag'
# saves
CLASS_SAVES = 'story__save'
# saves info
DATA_SAVES = 'aria-label'
# views
CLASS_VIEWS = 'story__views-count'
# views info
DATA_VIEWS = 'aria-label'
# text
CLASS_TEXT = 'story-block_type_text'

In [3]:
# Здесь необходимо сделать цикл, проходящий по всем файлам в папке, 
# постепенно распаршивающими страницу и записывающим результат в csv-файл.

In [16]:
def parse_story(story, atr):
    text = parse_text(story.find_all('div', class_=CLASS_TEXT))
    tags = parse_tags(story.find_all('a', class_=CLASS_TAG), DATA_TAG)

    saves = parse_saves(story.find(class_=CLASS_SAVES).attrs[DATA_SAVES])

    views_info = story.find(class_=CLASS_VIEWS)
    if views_info is None or views_info.get(CLASS_VIEWS) is None:
        views = 0
    else:
        views = parse_views(views_info.get(CLASS_VIEWS), DATA_AUTHOR_ID)


    if atr.get(DATA_RATING) is None:
        rating = 0
    else:
        rating = atr[DATA_RATING]

    article = Article(
        atr[DATA_ID], 
        story.find(class_=CLASS_TITLE).text, 
        atr[DATA_AUTHOR_ID],
        story.find(class_=CLASS_AUTHOR).attrs[DATA_AUTHOR_NAME],
        atr[DATA_COMMENTS_COUNT],
        rating,
        atr[DATA_META_RATING],
        atr[DATA_TIME],
        tags,
        views,
        saves,
        text)

    return article.get_json()

In [4]:
dir_name = 'pages'
files = os.listdir(dir_name)

In [12]:
def data_to_csv():
    with open('result.csv', 'w') as csvfile:
        fieldnames = ['id', 'title', 'author_id', 'author_name', 'comments', 'rating', 'rating_full', 'data', 'tags', 'views', 'saves', 'text']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for file in tqdm(files[0:10]):
            with open(os.path.join(dir_name, file), 'r', encoding='utf-8') as f:
                content = f.read()
                # парсинг страницы 
                soup = BeautifulSoup(content, 'lxml')
                stories = soup.find_all('article', class_='story')

                for story in stories:
                    atr = story.attrs
                    if is_ads(atr, DATA_AUTHOR_ID):
                        continue
                    else:
                        # print(parse_story(story, atr))
                        writer.writerow(parse_story(story, atr))

In [13]:
# 10 файлов, последний записанный - 3186
data_to_csv()

 30%|███       | 3/10 [00:00<00:00,  9.11it/s]

{
  "author_id": "1395543",
  "author_name": "MMaI",
  "comments": "6",
  "data": "1619407744",
  "id": "4442434",
  "rating": 0,
  "rating_full": "31:24",
  "saves": 0,
  "tags": [
    "[моё]",
    "Семья",
    "Брат",
    "Младший брат",
    "Мудрость",
    "Текст"
  ],
  "text": "  Мой младший брат, когда ему было 6-7 лет. Ситуация такова. Сидим За ужином семьей, я наполнил чайник водой, поставил его, сажусь за стол, тут мама говорит :\"кран закрой, вода капает. Да, ещё китайцы, очень мудрые люди, говорят :\"вода капает - деньги из дома уходят\". Все прониклись глубиной этой мысли, задумались, и тут малой выдаёт -\"а что тут мудрого? За воду просто платить нужно, вот и вся мудрость\"Все гораздо проще.",
  "title": "Китайская мудрость",
  "views": 0
}
{
  "author_id": "1413157",
  "author_name": "Tolstoknijnik",
  "comments": "8",
  "data": "1619407744",
  "id": "4442433",
  "rating": 0,
  "rating_full": "17:28",
  "saves": 1,
  "tags": [
    "Не мое",
    "Чтение",
    "Книги",
    

 60%|██████    | 6/10 [00:00<00:00,  8.11it/s]

{
  "author_id": "1048324",
  "author_name": "pisechkin",
  "comments": "17",
  "data": "1619356115",
  "id": "3659437",
  "rating": "35",
  "rating_full": "61:26",
  "saves": 1,
  "tags": [
    "[моё]",
    "Врачи",
    "Бюджетники",
    "Халатность",
    "Накипело",
    "Текст"
  ],
  "text": "  Мой сын в свои 4 года плохо разговаривал для своего возраста, не мог строить фразы и слова выговаривал не четко. Хотя знал весь алфавит и считал до 20-ти. Естественно в садике он не много отставал в развитии, потому как ему было сложно объяснить что либо. Была назначена комиссия по итогам которой было выписано направление в коррекционный садик. Нас с женой, как родителей, сей факт понятное дела никак не радовал и мы пошли к частному психологу, чтобы он еще раз все проверил. Только уже у него я понял насколько отличаются бюджетные врачи от частных.\nКогда была комиссия по моему ребенку, она проходила примерно так: сына попросили назвать пару фраз, он не смог и тут же было принято решение - реб

 70%|███████   | 7/10 [00:01<00:00,  6.07it/s]

...

{
  "author_id": "701377",
  "author_name": "Tak4toTytyVas",
  "comments": "6",
  "data": "1619217390",
  "id": "2801044",
  "rating": 0,
  "rating_full": "2:11",
  "saves": 0,
  "tags": [
    "Связной",
    "Банк",
    "Проблемммма",
    "Текст"
  ],
  "text": "  вопрос такой: есть ли на пикабу держатели карт связного банка ( пришлось в свое время приобрести) и есть ли у них проблемы с выполнением платежей и переводом средств. И решились ли они в конце концов? время московское. ",
  "title": "Вопрос",
  "views": 0
}
{
  "author_id": "538990",
  "author_name": "roost",
  "comments": "7",
  "data": "1619217390",
  "id": "2801041",
  "rating": 0,
  "rating_full": "5:23",
  "saves": 0,
  "tags": [
    "Любовь",
    "Алена",
    "Праздники",
    "Текст"
  ],
  "text": "  Ребят, помогите пожалуйста, сегодня у меня с девушкой праздник, но я бомж и у меня нету денег:( напишите кому не жалко пару рублей ей смс по типу того что я её поздравляю и люблю:) номер +7 (963) 975-67-93",
  "title": "По

 90%|█████████ | 9/10 [00:01<00:00,  4.59it/s]

{
  "author_id": "464821",
  "author_name": "degra",
  "comments": "1",
  "data": "1619163226",
  "id": "1981936",
  "rating": 0,
  "rating_full": "3:16",
  "saves": 0,
  "tags": [
    "Украина",
    "Киев",
    "Евромайдан",
    "Текст"
  ],
  "text": "  С утра начались противостояния в нескольких точках.\nЕсть пострадавшие\nhttp://ua-selector.in/",
  "title": "В Киеве опять дым, стрельба и взрывы",
  "views": 0
}
{
  "author_id": "390265",
  "author_name": "Akbarbabar",
  "comments": "11",
  "data": "1619163226",
  "id": "1981911",
  "rating": 0,
  "rating_full": "13:26",
  "saves": 0,
  "tags": [
    "[моё]",
    "DayZ",
    "Rust",
    "Текст"
  ],
  "text": "  Такое чувство что скоро поклонники Dayz и Rust , будут обсирать друг друга, как поклонники Xbox и PS",
  "title": "Dayz vs Rust",
  "views": 0
}
{
  "author_id": "323008",
  "author_name": "stydeht",
  "comments": "3",
  "data": "1619163226",
  "id": "1981909",
  "rating": "18",
  "rating_full": "47:29",
  "saves": 0,
  "tag

100%|██████████| 10/10 [00:02<00:00,  4.57it/s]

{
  "author_id": "404867",
  "author_name": "Limittum",
  "comments": "5",
  "data": "1619179409",
  "id": "2286169",
  "rating": 0,
  "rating_full": "3:14",
  "saves": 0,
  "tags": [
    "[моё]",
    "Футбол",
    "Спортивные прогнозы",
    "Прогнозы на футбол",
    "Текст"
  ],
  "text": "  Доброго времени суток, друзья!\n\nСегодня понедельник, а это значит, что пора подводить итоги очередного тура.\n\nРезультаты матчей:\n\n1\tАмкар\t2:2\tКраснодар\n2\tАнжи\t0:1\tКрылья Советов\n3\tСпартак\t3:2\tДинамо\n4\tКубань\t1:4\tЗенит\n5\tРубин\t1:1\tТерек\n6\tЦСКА\t1:0\tЛокомотив\n7\tТомь\t3:2\tРостов\n8\tВолга\t1:2\tУрал\n9\tВаленсия\t2-1\tСельта\n10\tРеал Мадрид\t3-1\tЭспаньол\n11\tБарселона\t1-1\tАтлетико М\n12\tСевилья\t3-1\tЭльче\n13\tЮвентус\t3-0\tКальяри\n14\tНаполи\t5-1\tВерона\n15\tКьево\t2-1\tИнтер\n16\tМилан\t2-1\tСассуоло\n17\tДженоа\t1-0\tРома\n18\tПСЖ\t4-0\tМонпелье\n19\tМонако\t1-1\tБордо\n\n\n\nПятерка лучших по итогам 1-го тура:\n\n1\tSDnRnR - 24 очка\n2\tMofa\t - 19 очков\n3